In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from dataset import ToxicCommentsDataset

/mnt/d/venv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


data_dir = '/home/nhan/Downloads/toxic_comments'
<br>
data_dir = '/mnt/d/kaggle-toxic-comments/data'

In [2]:
toxic_comments_dataset = ToxicCommentsDataset('/mnt/d/kaggle-toxic-comments/data',
                                              'train.csv',
                                              'test.csv')

In [3]:
train_texts, y_train, test_texts = toxic_comments_dataset.get_texts_and_train_labels()

In [4]:
x_train, x_test = toxic_comments_dataset.tokenize_by_keras(max_words=100000, maxlen=150)

Found 210337 unique tokens in train corpus.
Found 394787 unique tokens in test corpus.


In [20]:
tf_idf = TfidfVectorizer()
xgb = XGBClassifier(silent=False)

In [9]:
tf_idf_x_train = tf_idf.fit_transform(train_texts)
tf_idf_x_test = tf_idf.fit_transform(test_texts)

In [23]:
tf_idf_x_train = tf_idf_x_train[:, :100]

In [16]:
param_grid = {'n_estimators': np.array([5, 10, 15, 20, 25]),
              'max_depth': np.array([5,10,15,20,25]),
              'subsample': np.array([0.5,0.6,0.7,0.8,0.9,1.0]),
              'colsample_bytree': np.array([0.5,0.6,0.7,0.8,0.9,1.0]),
              'learning_rate': np.array([0.01,0.05,0.10,0.20,0.30,0.40]),
              'gamma': np.array([0.00,0.05,0.10,0.15,0.20]),
              'scale_pos_weight': np.array([30,40,50,300,400,500,600,700])}

In [44]:
randomized = RandomizedSearchCV(xgb, param_distributions=param_grid,
                                n_iter=5, scoring='roc_auc')

In [45]:
randomized.fit(tf_idf_x_train, y_train[:, 0])

RandomizedSearchCV(cv=None, error_score='raise',
          estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=False, subsample=1),
          fit_params=None, iid=True, n_iter=5, n_jobs=1,
          param_distributions={'gamma': array([0.  , 0.05, 0.1 , 0.15, 0.2 ]), 'n_estimators': array([ 5, 10, 15, 20, 25]), 'scale_pos_weight': array([ 30,  40,  50, 300, 400, 500, 600, 700]), 'learning_rate': array([0.01, 0.05, 0.1 , 0.2 , 0.3 , 0.4 ]), 'colsample_bytree': array([0.5, 0.6, 0.7, 0.8, 0.9, 1. ]), 'max_depth': array([ 5, 10, 15, 20, 25]), 'subsample': array([0.5, 0.6, 0.7, 0.8, 0.9, 1. ])},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
         

In [ ]:
from sklearn.model_selection import 

In [3]:
outputs = pd.read_csv('outputs/xgb_outputs.csv')
outputs.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0.070621,0.036615,0.046710,0.000242,0.085841,0.422239
1,0.064460,0.092891,0.044754,0.000390,0.055833,0.425043
2,0.074079,0.024515,0.037966,0.000172,0.108841,0.417938
3,0.045645,0.023171,0.045993,0.000193,0.030483,0.417684
4,0.897002,0.235093,0.545828,0.001552,0.815214,0.465578


In [4]:
toxic_comments_dataset = ToxicCommentsDataset(data_dir='/home/nhan/Downloads/toxic_comments',
                                              train_csv_file='train.csv',
                                              test_csv_file='test.csv')

In [5]:
train_texts, y_train, test_texts = toxic_comments_dataset.get_texts_and_train_labels()

In [12]:
predictions_on_test = outputs.iloc[len(train_texts):, :]

In [13]:
len(predictions_on_test) == len(test_texts)

True

In [16]:
test_idx = pd.read_csv(os.path.join('/home/nhan/Downloads/toxic_comments', 'test.csv')).iloc[:, 0]

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [22]:
columns = predictions_on_test.columns.tolist()
columns = columns[-1:] + columns[:-1]
predictions_on_test = predictions_on_test[columns]

In [17]:
predictions_on_test.loc[:, 'id'] = test_idx.values
predictions_on_test.to_csv('outputs/xgb_outputs_test_with_idx.csv', index=False)

In [18]:
predictions_on_train = outputs.iloc[:len(train_texts), :]
len(predictions_on_train) == len(train_texts)

True

In [19]:
train_idx = pd.read_csv(os.path.join('/home/nhan/Downloads/toxic_comments', 'train.csv')).iloc[:, 0]

In [20]:
predictions_on_train.loc[:, 'id'] = train_idx.values
predictions_on_train = predictions_on_train[columns]
predictions_on_train.to_csv('outputs/xgb_outputs_train_with_idx.csv', index=False)

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
